In [64]:
import pandas as pd

In [77]:
movies=pd.read_csv(r"C:\Users\DELL\Downloads\archive (1)\movies.csv")
ratings=pd.read_csv(r"C:\Users\DELL\Downloads\archive (1)\ratings.csv")

In [66]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [67]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [68]:
# Calculate average rating for each movie
movie_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()

In [69]:
# Merge with movie titles
popularity_df = pd.merge(movie_ratings, movies[['movieId', 'title']], on='movieId', how='left')

In [70]:
# Sort by average rating in descending order
popularity_df = popularity_df.sort_values(by='rating', ascending=False)

In [71]:
# Display top N popular movies
top_n = 10
top_movies = popularity_df.head(top_n)
print(f"Top {top_n} Popular Movies:")
print(top_movies[['title', 'rating']])

Top 10 Popular Movies:
                                          title  rating
7638      Paper Birds (Pájaros de papel) (2010)     5.0
8089                 Act of Killing, The (2012)     5.0
9065                            Jump In! (2007)     5.0
9076                               Human (2015)     5.0
9078                        L.A. Slasher (2015)     5.0
4245                           Lady Jane (1986)     5.0
8136             Bill Hicks: Revelations (1993)     5.0
8130               Justice League: Doom (2012)      5.0
4240  Open Hearts (Elsker dig for evigt) (2002)     5.0
9104                     Formula of Love (1984)     5.0


## Content Base

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [73]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'].fillna(''))

In [74]:
# Calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [75]:
def content_based_recommendations(movie_title, cosine_sim=cosine_sim, movies=movies):
    # Check if the movie title exists in the dataset
    if movie_title not in movies['title'].values:
        return f"Movie '{movie_title}' not found in the dataset."

    idx = movies.index[movies['title'] == movie_title].tolist()
    
    # Check if the movie title was found
    if not idx:
        return f"Movie '{movie_title}' not found in the dataset."
    
    idx = movies.index[movies['title'] == movie_title].tolist()

    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10 similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [79]:
# Test the content-based recommender
movie_title = 'Dark Knight, The (2008)'
content_recommendations = content_based_recommendations(movie_title)
print(f"\nContent-Based Recommendations for '{movie_title}':")
print(content_recommendations)


Content-Based Recommendations for 'Dark Knight, The (2008)':
8387                                Need for Speed (2014)
5917                                 Batman Begins (2005)
7599       Fast Five (Fast and the Furious 5, The) (2011)
6852                                     Eagle Eye (2008)
8103                       Good Day to Die Hard, A (2013)
8166    Fast & Furious 6 (Fast and the Furious 6, The)...
8149            Grandmaster, The (Yi dai zong shi) (2013)
7768                        Dark Knight Rises, The (2012)
8233                                Man of Tai Chi (2013)
8201                              White House Down (2013)
Name: title, dtype: object
